In [1]:
import streamlit  as st
import json 
import os
import pandas as pd
import numpy as np
import mysql.connector as mc
connection = mc.connect(user= 'root',password='ss3112',host = 'localhost')
cursor = connection.cursor()
cursor.execute('CREATE DATABASE phonepe_pulse')
cursor.execute('USE phonepe_pulse')


In [2]:
#                          1. This is to direct the path to get the data as states

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


path = "pulse/data/aggregated/transaction/country/india/state/"
user_state_list = os.listdir(path)

# Agg_state_list--> to get the list of states in India


#-----------------------------------------------------------------------------------------------------------------------

# This is to extract the data's to create a dataframe

clm = {'State': [], 'Year': [], 'Quater': [], 'Transaction_type': [],
       'Transaction_count': [], 'Transaction_amount': []}
for i in user_state_list:
    p_i = path+i+"/"
    Agg_yr = os.listdir(p_i)
    for j in Agg_yr:
        p_j = p_i+j+"/"
        Agg_yr_list = os.listdir(p_j)
        for k in Agg_yr_list:
            p_k = p_j+k
            Data = open(p_k, 'r')
            D = json.load(Data)
            try:
                for z in D['data']['transactionData']:
                    Name = z['name']
                    count = z['paymentInstruments'][0]['count']
                    amount = z['paymentInstruments'][0]['amount']
                    clm['Transaction_type'].append(Name)
                    clm['Transaction_count'].append(count)
                    clm['Transaction_amount'].append(amount)
                    clm['State'].append(i)
                    clm['Year'].append(j)
                    clm['Quater'].append(int(k.strip('.json')))
            except:
                pass

Agg_Transac = pd.DataFrame(clm)
# Succesfully created a dataframe
#Agg_Transac.to_csv('phonepe_csv/Agg_Transac.csv',index = False)
#Agg_Transac_csv = pd.read_csv('phonepe_csv/Agg_Transac.csv')
Agg_Transac

,State,Year,Quater,Transaction_type,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,andaman-&-nicobar-islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,andaman-&-nicobar-islands,2018,1,Merchant payments,298,4.525072e+05
3,andaman-&-nicobar-islands,2018,1,Financial Services,33,1.060142e+04
4,andaman-&-nicobar-islands,2018,1,Others,256,1.846899e+05
...,...,...,...,...,...,...
3589,west-bengal,2022,4,Peer-to-peer payments,184380244,6.202222e+11
3590,west-bengal,2022,4,Merchant payments,171667404,1.408077e+11
3591,west-bengal,2022,4,Recharge & bill payments,48921147,2.602663e+10
3592,west-bengal,2022,4,Financial Services,268388,2.611229e+08


In [ ]:
cursor.execute("create table aggregate_transaction(state varchar(45), year int, Quater int, tansaction_type varchar(45), transaction_count int, transaction_amount float)")
for record in Agg_Transac.values:
    cursor.execute("insert into aggregate_transaction(state, year, Quater, tansaction_type, transaction_count, transaction_amount) values{}".format(tuple(record)))
    connection.commit()
print('Data inserted into Table-1 successfully')
    

#**************************************** ------  1 st finished  ------************************************************
    